In [32]:
import csv
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chromedriver_path = r'C:\Users\luisf\Desktop\MDS_thesis\scraper_gn\chromedriver-win64\chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [33]:
search_query = "amlo dice"
start_date = "2018-12-03"
end_date = "2024-09-22"

url = f'https://www.google.com/search?q={search_query}+after:{start_date}+before:{end_date}&tbm=nws'

driver.get(url)


In [39]:
# Initialize an empty list to hold the results
news_data = []

# Set a limit for the number of articles
limit = 100
count = 0
current_page = 1

while current_page <= max_pages:
    print(f"Scraping page {current_page}")
    
    # Wait for the news results to load
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.SoaBEf')))
    
    # Select all news article blocks by targeting the SoaBEf class
    news_results = driver.find_elements(By.CSS_SELECTOR, 'div.SoaBEf')
    
    for news_div in news_results:
        if count >= limit:
            break  # Stop if we've reached the limit

        news_item = []
        try:
            # Extract the link to the article
            news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')

            # Extract the title of the article (div with role="heading")
            news_title = news_div.find_element(By.CSS_SELECTOR, 'div[role="heading"]').text

            # Extract the source domain (usually at the beginning of the div)
            news_domain = news_div.find_element(By.CSS_SELECTOR, 'span.XTjFC').text

            # Extract the description (after the heading div)
            news_description = news_div.find_element(By.CSS_SELECTOR, 'div > div > div:nth-child(2)').text

            # Extract the date
            news_date = news_div.find_element(By.CSS_SELECTOR, 'span.WG9SHc').text

            # Append the collected information to the list
            news_data.append([news_link, news_domain, news_title, news_description, news_date])
            count += 1  # Increment the count of news articles

        except Exception as e:
            print("Error scraping:", e)

    # Try to locate the "Next" button and click it
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'a#pnnext')  # Google uses "pnnext" for the next page button
        next_button.click()
        current_page += 1
        time.sleep(3)  # Adding a short sleep to wait for the next page to load
    except Exception as e:
        print("No more pages or error:", e)
        break  # Exit the loop if there's no next button or an error occurs
            


In [40]:
print(news_data)
print(len(news_data))

[['https://www.bbc.com/mundo/articles/cv22e6g3x59o', 'BBC\nAMLO: los 4 pilares que explican su alta popularidad tras 6 años gobernando México (y cómo lo ven sus críticos)\nA cuatro meses del fin de su mandato, el presidente de México, Andrés Manuel López Obrador, tiene una aprobación cercana al 60%, una cifra que muchos...\n.\n29 may 2024', 'AMLO: los 4 pilares que explican su alta popularidad tras 6 años gobernando México (y cómo lo ven sus críticos)', 'A cuatro meses del fin de su mandato, el presidente de México, Andrés Manuel López Obrador, tiene una aprobación cercana al 60%, una cifra que muchos...', 'BBC'], ['https://www.dw.com/es/amlo-no-reconoce-triunfo-de-maduro-y-dice-esperar-las-actas/a-70036261', 'DW\nAMLO no reconoce triunfo de Maduro y dice esperar las actas\nEl gobernante mexicano no se ha sumado al rechazo conjunto que el viernes (23.8. 2024) hicieron Chile, Argentina, Costa Rica, Ecuador, Estados Unidos, Guatemala...\n.\nhace 1 mes', 'AMLO no reconoce triunfo de Madur

In [36]:
df = pd.DataFrame(news_data, columns=["Link", "Domain", "Title", "Description", "Date"])
print(df)
print(len(df))

                                              Link Domain  \
0  https://www.bbc.com/mundo/articles/cv22e6g3x59o    BBC   

                                               Title  \
0  AMLO: los 4 pilares que explican su alta popul...   

                                         Description         Date  
0  A cuatro meses del fin de su mandato, el presi...  29 may 2024  
1


In [ ]:

# Save the results to a CSV file
csv_file_path = "news_results.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(["Link", "Domain", "Title", "Description", "Date"])
    # Write the news data
    writer.writerows(news_data)

driver.quit()

print(f"Results saved to {csv_file_path}")